In [ ]:
#Import libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Twitter credentials for the app
consumerkey = '****************************'
consumersecret = '****************************'
accesstoken= '****************************'
accesstokensecret = '****************************'

In [ ]:
#Create authentication object
authenticate = tweepy.OAuthHandler(consumerkey, consumersecret)

#Create access and access secret token
authenticate.set_access_token(accesstoken, accesstokensecret)

#Create API object
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [ ]:
#Extract 200 tweets from Donald Trump
posts = api.user_timeline (screen_name = "RealDonaldTrump", count = 200, lang="en", tweet_mode= "extended")
#tweet mode extended - full text of tweet

#Print the last 5 tweets 
print("Show the 5 recent tweets: \n")
i=1
for tweet in posts[0:5]:
    print(str(i) + '. ' +tweet.full_text + '\n')
    i=i+1

In [ ]:
#Create dataframe
df=pd.DataFrame( [tweet.full_text for tweet in posts], columns = ['Tweets'])

#See first 5 rows
df.head()

In [ ]:
#Clean the data
def cleanTxT(text):
    text = re.sub(r'@[A-Za-z0-9_:]+','', text)   #Removes @ mentions (the r tells python it is a raw string)
    text=re.sub(r'#','',text)    #Removes hashes
    text=re.sub(r'LIVE','', text)   #Removes LIVE
    text=re.sub(r'NOW','', text)    #Removes NOW
    text=re.sub(r'RT[\s]+', '', text)    #Removes RT
    text=re.sub(r'https?:\/\/\S+','',text)    #Removes URLs
    return text


df['Tweets'] = df['Tweets'].apply(cleanTxT)

df.head()
    

In [ ]:
#Create function to get subjectivity 
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create function to get polarity 
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#Create new cols
df['Subjectivity']=df['Tweets'].apply(getSubjectivity)
df['Polarity']=df['Tweets'].apply(getPolarity)

df.head()

In [ ]:
#Wordcloud
allWords=' '.join([twts for twts in df['Tweets']])
wordCloud = WordCloud(width=5000, height=3000, random_state=21, max_font_size=1500).generate(allWords)

plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
#Compute negative, positive, neutral analyses
def getAnalysis(score):
    if score<0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'
    
df['Analysis']=df['Polarity'].apply(getAnalysis)

df.head()

In [ ]:
#Print all positive tweets
j=1
sortedDF= df.sort_values(by=['Polarity'])
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Analysis'][i]=='Positive'):
        print(str(j) + '. ' +sortedDF['Tweets'][i])
        print()
        j=j+1

In [ ]:
#Print all negative tweets
j=1
sortedDF= df.sort_values(by=['Polarity'], ascending=False)
for i in range(0,sortedDF.shape[0]):
    if (sortedDF['Analysis'][i]=='Negative'):
        print(str(j) + '. ' +sortedDF['Tweets'][i])
        print()
        j=j+1

In [ ]:
#Plot the Polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
    plt.scatter(df['Polarity'][i],df['Subjectivity'][i],color='Blue')
    
    
plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()
    

In [ ]:
#Percentage of Positive Tweets
ptweets = df[df.Analysis == 'Positive']
ptweets = ptweets['Tweets']

ptweets.head()

In [ ]:
round((ptweets.shape[0]/df.shape[0])*100,1)

In [ ]:
#Show the value counts
df['Analysis'].value_counts()

#Visualise counts
plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show()